In [1]:
!pip install --quiet arcgis

In [2]:
from arcgis.gis import GIS;
from arcgis.features import FeatureLayer,Table;
import requests;

In [16]:
huc12_url = 'https://watersgeo.epa.gov/arcgis/rest/services/Support/HydrologicUnits/MapServer';
huc12_lyr = FeatureLayer(huc12_url + '/6');

huc12s = huc12_lyr.query(
    where = '1=1',
    out_fields = 'huc12',
    return_geometry = False,
    return_all_records = False,
    result_record_count = 1000,
    result_offset = 2000
);

huc12s.sdf.count()

OBJECTID    1000
huc12       1000
dtype: int64

In [18]:
attains_url = 'https://attains.epa.gov/attains-public/api/huc12summary';
agol_url    = 'https://services1.arcgis.com/BN5Mj6q47j71sqyJ/arcgis/rest/services/ATTAINS_Assessments/FeatureServer/0/query';

for index, row in huc12s.sdf.iterrows():
    response = requests.get(attains_url,params={'huc': row.huc12});
    resp_json = response.json();
    assessments = resp_json["items"][0]["assessmentUnits"];
    
    if len(assessments) > 0:
       qry = 'assessmentunitidentifier%20in%20(';
       for item in assessments:
            qry = qry + '%27' + item["assessmentUnitId"] + '%27%2C%20'
            
       qry = qry.rstrip('%2C%20') + ")"
    
       bam_url = agol_url + '?f=pbf&outFields=*&spatialRel=esriSpatialRelIntersects&where=' + qry + '&ondev=1'
    
       response = requests.get(bam_url);
    
       if response.status_code != 200:
           raise RuntimeError(response.request.url);
    
       print(row.huc12 + " " + str(len(assessments)) + " Assessments - " + response.headers["x-cache"])
        
    else:
       print(row.huc12 + " No Assessments")
    

SyntaxError: invalid syntax (<ipython-input-18-8ed580ab0f50>, line 23)